In [3]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import copy



In [4]:
f= open("parameters.txt","r")
n= int(f.readline())
print(n)
transitionMatrix= np.empty([n,n])
for  i in range(n):
    str = f.readline()
    l=[np.double(p) for p in str.split()]
    transitionMatrix[i] = l
# transitionMatrix= np.asmatrix(transitionMatrix)

Mean= f.readline()
print( "mean is ",Mean)
Mean=[np.double(p) for p in Mean.split()]
Variance = f.readline()
Variance=[np.double(p) for p in Variance.split()]
transitionMatrix

2
mean is  200	100



array([[0.7, 0.3],
       [0.1, 0.9]])

In [5]:
def initialEstimator(mat):
    mat1= copy.deepcopy(mat)   
    for i in range(20):
        mat1= mat1 @ mat1
    # print(np.isclose(mat1, mat1[0]).all())
    print(mat1)
    return mat1[0]


initial= initialEstimator(transitionMatrix)



[[0.7 0.3]
 [0.1 0.9]]
[[0.25 0.75]
 [0.25 0.75]]


In [6]:
dataFile = open("data.txt","r")
data= np.double(dataFile.readlines())

dataSize = len(data)
emissionMatrix=np.empty((n,dataSize))
for i in range(n):
    emissionMatrix[i]= norm(Mean[i],np.sqrt(Variance[i])).pdf(data)

emissionMatrix=np.asmatrix(emissionMatrix).T
emissionMatrix.shape
np.savetxt("emissionMatrix.txt",emissionMatrix,delimiter=' ')


In [82]:
dpTable= np.zeros([dataSize,n]) #dpTable size 2x1000
backTrack = np.zeros([dataSize,n])
for i in range(n):
    dpTable[0,i]=np.log(initial[i]) + np.log( emissionMatrix[0,i])
for i in range(1,dataSize):
    for  s in range (n):
        max=-np.Infinity
        prev=-1
        for k in range(n):
            temp= dpTable[i-1,k] + np.log(emissionMatrix[i,s]) + np.log(transitionMatrix[k,s])
            if ( temp > max) : 
                max = temp
                prev= k        
        dpTable[i][s] = max
        backTrack[i][s]= int(prev)

ans =[]
ansIndex = int(np.argmax(np.array(dpTable[dataSize-1])))
ans.append(ansIndex)
for  i in range(dataSize-2,0,-1):
    # print(i ,ansIndex)
    ansIndex= backTrack[i,int(ansIndex)]
    ans.append(ansIndex)

ans= np.array(ans)
ans =np.flip(ans)
# print("after flip" ,ans)
ans= ans.tolist()

# print(ans)
for i in range(len(ans)):
    if ans[i]==0:
        ans[i]="El Nino"
    else :
        ans[i]= "La Nina"
np.savetxt("dpTable.txt",dpTable,delimiter=' ')
# np.savetxt("vitebriOutput.txt",ans,delimiter="\n")
with open('vitebriOutput.txt', 'w') as f:
    for item in ans:
        f.write("\"%s\"\n" % item)

        



[-68295.49751944 -67780.52661432]


In [60]:
def forwardCalc(emission,transition,initial):
    forward = np.zeros([dataSize,n])
    for i in range(n):
        forward[0,i]= initial[i]*emission[0,i]
    
    forward[0]= forward[0]/np.sum(forward[0])
    
    for i in range(1,dataSize):
        for j in range(n):
            forward[i,j]= (forward[i-1] @ transition[:,j] ) *emission[i,j]
        forward[i]= forward[i]/np.sum(forward[i])
    np.savetxt("forward.txt",forward,delimiter=" ")

    return forward


array([[1.05803429e-198, 1.00000000e+000],
       [3.12963336e-226, 1.00000000e+000],
       [1.30526483e-213, 1.00000000e+000],
       ...,
       [5.31489971e-006, 9.99994685e-001],
       [1.00117350e-235, 1.00000000e+000],
       [2.24376601e-224, 1.00000000e+000]])

In [61]:
def backwardCalc(emission,transition) :
    backward= np.zeros([dataSize,n])
    backward[dataSize-1]=np.ones(n)
    
    for i in range(dataSize-2,-1,-1):
        for j in range(n):
            temp= copy.deepcopy(backward[i+1])
            for k in range(n):
                temp[k]= temp[k]*emission[i+1,k]
            # backward[i,j]= (backward[i+1] * emission[i+1,:])@transition[j,:] 
            backward[i,j]= temp@transition[j,:] 

        backward[i]= backward[i]/np.sum(backward[i])
    np.savetxt("backwardTemp.txt",backward,delimiter=" ")
    return backward
    


array([[0.25, 0.75],
       [0.25, 0.75],
       [0.25, 0.75],
       ...,
       [0.25, 0.75],
       [0.25, 0.75],
       [1.  , 1.  ]])

In [93]:
forward = forwardCalc(emissionMatrix,transitionMatrix,initial)
backward = backwardCalc(emissionMatrix,transitionMatrix)

piStar =np.zeros([dataSize,n])
for i in range(dataSize):
    for  j in range(n):
        piStar[i,j]=forward[i,j]* backward[i,j]
    piStar[i]=piStar[i]/np.sum(piStar[i])


meanList= np.zeros(n)
SDlist=np.zeros(n)
for i in range(n):
    mul = piStar[:,i] *data
    sum = np.sum(piStar[:,i])
    meanList[i]= np.sum(mul)/sum
print(meanList)

for i in range(n):
    mul = piStar[:,i] *(data-meanList[i])**2
    sum = np.sum(piStar[:,i])
    SDlist[i]= np.sqrt(np.sum(mul)/sum)
print(SDlist)


[151.91073271 118.37430848]
[ 1.37402556 23.45636704]


In [90]:
piDoubleStar = np.zeros([dataSize-1,n,n])
for i in range(dataSize-1):
    for k in range(n):
        for l in range(n):
            piDoubleStar[i,k,l]=forward[i,k] * backward[i+1,l] * emissionMatrix[i+1,l]* transitionMatrix[k,l]
    piDoubleStar[i]=piDoubleStar[i]/np.sum(piDoubleStar[i])
# np.savetxt("piDoubleStar.txt",piDoubleStar,delimiter=" ")

# for  element in piDoubleStar :
#     print(element)

print(piDoubleStar[1,1,1])  
ans=np.zeros([n,n])
for i in range(dataSize-1):
    for  k in range(n):
        for l in range(n):
            ans[k,l]=ans[k,l]+piDoubleStar[i,k,l]

for i in range(n):
    ans[i]= ans[i]/np.sum(ans[i])
print(ans)



1.0
[[0.46127025 0.53872975]
 [0.22031122 0.77968878]]
